In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## pt -> onnx -> trt

### pytoch model to onnx model

In [ ]:
!pip install onnx

In [9]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
import torch.onnx


model = torchvision.models.resnet50(weights = "ResNet50_Weights.IMAGENET1K_V2")
model.eval()
# Input to the model
batch_size = 1
x = torch.randn(batch_size, 3, 224, 224, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(model,# model being run
        x,   # model input (or a tuple for multiple inputs)
        "resnet50.onnx",  # where to save the model (can be a file or file-like object)
        export_params=True, # store the trained parameter weights inside the model file
        opset_version=10,   # the ONNX version to export the model to
        do_constant_folding=True, # whether to execute constant folding for optimization
        input_names = ['input'],  # the model's input names
        output_names = ['output'], # the model's output names
        dynamic_axes={'input' : {0 : 'batch_size'}, # variable length axes
              'output' : {0 : 'batch_size'}})

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



### onnx model to trt engine

In [10]:
!python3 -m pip install --upgrade tensorrt

In [11]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
def build_engine(onnx_path, shape = [1,224,224,3]):

   """
   This is the function to create the TensorRT engine
   Args:
      onnx_path : Path to onnx_file.
      shape : Shape of the input of the ONNX file.
  """
   with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, builder.create_builder_config() as config, trt.OnnxParser(network, TRT_LOGGER) as parser:
       config.max_workspace_size = (256 << 20)
       with open(onnx_path, 'rb') as model:
           parser.parse(model.read())
       network.get_input(0).shape = shape
       engine = builder.build_engine(network, config)
       return engine

def save_engine(engine, file_name):
   buf = engine.serialize()
   with open(file_name, 'wb') as f:
       f.write(buf)
def load_engine(trt_runtime, plan_path):
   with open(plan_path, 'rb') as f:
       engine_data = f.read()
   engine = trt_runtime.deserialize_cuda_engine(engine_data)
   return engine

In [12]:
import argparse
from onnx import ModelProto
import tensorrt as trt

engine_name = "resnet50.trt"
onnx_path = "./resnet50.onnx"
batch_size = 1

model = ModelProto()
with open(onnx_path, "rb") as f:
    model.ParseFromString(f.read())

d0 = model.graph.input[0].type.tensor_type.shape.dim[1].dim_value
d1 = model.graph.input[0].type.tensor_type.shape.dim[2].dim_value
d2 = model.graph.input[0].type.tensor_type.shape.dim[3].dim_value
shape = [batch_size , d0, d1 ,d2]
engine = build_engine(onnx_path, shape= shape)
save_engine(engine, engine_name)

<ipython-input-11-d5c2a7eeabb5>:14: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = (256 << 20)
<ipython-input-11-d5c2a7eeabb5>:18: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


### trt inference

In [ ]:
!pip install pycuda
!wget https://www.akc.org/wp-content/uploads/2021/07/Cavalier-King-Charles-Spaniel-laying-down-indoors.jpeg -O ./dog.jpeg
!wget https://www.collinsdictionary.com/images/full/hen_151155842.jpg -O ./hen.jpg

In [14]:
import tensorrt as trt
import pycuda.driver as cuda
import numpy as np
import pycuda.autoinit

def allocate_buffers(engine, batch_size, data_type):

   """
   This is the function to allocate buffers for input and output in the device
   Args:
      engine : The path to the TensorRT engine.
      batch_size : The batch size for execution time.
      data_type: The type of the data for input and output, for example trt.float32.

   Output:
      h_input_1: Input in the host.
      d_input_1: Input in the device.
      h_output_1: Output in the host.
      d_output_1: Output in the device.
      stream: CUDA stream.

   """

   # Determine dimensions and create page-locked memory buffers (which won't be swapped to disk) to hold host inputs/outputs.
   h_input_1 = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(0)), dtype=trt.nptype(data_type))
   h_output = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(1)), dtype=trt.nptype(data_type))
   # Allocate device memory for inputs and outputs.
   d_input_1 = cuda.mem_alloc(h_input_1.nbytes)

   d_output = cuda.mem_alloc(h_output.nbytes)
   # Create a stream in which to copy inputs/outputs and run inference.
   stream = cuda.Stream()
   return h_input_1, d_input_1, h_output, d_output, stream

def load_images_to_buffer(pics, pagelocked_buffer):
   preprocessed = np.asarray(pics).ravel()
   np.copyto(pagelocked_buffer, preprocessed)

def do_inference(engine, pics_1, h_input_1, d_input_1, h_output, d_output, stream, batch_size, height, width):
   """
   This is the function to run the inference
   Args:
      engine : Path to the TensorRT engine
      pics_1 : Input images to the model.
      h_input_1: Input in the host
      d_input_1: Input in the device
      h_output_1: Output in the host
      d_output_1: Output in the device
      stream: CUDA stream
      batch_size : Batch size for execution time
      height: Height of the output image
      width: Width of the output image

   Output:
      The list of output images

   """

   load_images_to_buffer(pics_1, h_input_1)

   with engine.create_execution_context() as context:
      # Transfer input data to the GPU.
      cuda.memcpy_htod_async(d_input_1, h_input_1, stream)

      # Run inference.

      context.profiler = trt.Profiler()
      context.execute(batch_size=1, bindings=[int(d_input_1), int(d_output)])

      # Transfer predictions back from the GPU.
      cuda.memcpy_dtoh_async(h_output, d_output, stream)
      # Synchronize the stream
      stream.synchronize()
      # Return the host output.
      # out = h_output.reshape((batch_size, -1, height, width))
      out = h_output
      return out.reshape(1, -1)
def softmax(x):
  # x -= np.max(x , axis=1 , keepdims = True)
  x = np.exp(x) / np.sum(np.exp(x) , axis=1 , keepdims = True)
  return x

In [30]:
# import keras
import tensorrt as trt
import numpy as np
from PIL import Image
import tensorrt as trt
import torch.nn as nn
# import labels  # from cityscapes evaluation script
# import skimage.transform

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

input_file_path = "hen.jpg"
serialized_plan_fp32 = "resnet50.trt"
HEIGHT = 224
WIDTH = 224

img = Image.open(input_file_path).resize((WIDTH , HEIGHT))
# img.show()
img = np.asarray(img)
im = np.array(img, dtype=np.float32, order='C')
im = im.transpose((2, 0, 1))
im = (2.0 / 255.0) * im - 1.0

engine = load_engine(trt_runtime, serialized_plan_fp32)
h_input, d_input, h_output, d_output, stream = allocate_buffers(engine, 1, trt.float32)


out = do_inference(engine, im, h_input, d_input, h_output, d_output, stream, 1, HEIGHT, WIDTH)
out = softmax(out)

label = np.argmax(out)
value = np.max(out)

print(label , value)

8 0.5181372


<ipython-input-14-fc77fe6cee0c>:25: DeprecationWarning: Use get_tensor_shape instead.
  h_input_1 = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(0)), dtype=trt.nptype(data_type))
<ipython-input-14-fc77fe6cee0c>:26: DeprecationWarning: Use get_tensor_shape instead.
  h_output = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(1)), dtype=trt.nptype(data_type))
<ipython-input-14-fc77fe6cee0c>:68: DeprecationWarning: Use execute_v2 instead.
  context.execute(batch_size=1, bindings=[int(d_input_1), int(d_output)])
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: pagelocked_host_allocation in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspe

## torch2trt

In [ ]:
!pip install onnxruntime
!wget https://www.akc.org/wp-content/uploads/2021/07/Cavalier-King-Charles-Spaniel-laying-down-indoors.jpeg -O ./dog.jpeg
!wget https://www.collinsdictionary.com/images/full/hen_151155842.jpg -O ./hen.jpg

In [ ]:
!python3 -m pip install --upgrade tensorrt
!git clone https://github.com/NVIDIA-AI-IOT/torch2trt
%cd torch2trt
!python setup.py install
# !cmake -B build . && cmake --build build --target install && ldconfig
%cd ..


#### 要重啟才能使用

In [ ]:
import os
os._exit(00)

In [2]:
import torch
from torch2trt import torch2trt
import torchvision


# create some regular pytorch model...
model = torchvision.models.resnet50(weights = "ResNet50_Weights.IMAGENET1K_V2")
model.eval().cuda()
# Input to the model
batch_size = 1
x = torch.randn(batch_size, 3, 224, 224, requires_grad=True).cuda()

# convert to TensorRT feeding sample data as input
model_trt = torch2trt(model, [x])


In [5]:
from torch import nn
from PIL import Image
from torchvision import transforms

input_file_path = "hen.jpg"
img = Image.open(input_file_path)

tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

img = tfm(img).cuda()

pred = model_trt(img).to("cpu")
softmax = nn.Softmax(dim=1)
pred = softmax(pred)
# print(pred)
label = torch.argmax(pred)
conf = torch.max(pred)
print(label , conf)

tensor(8) tensor(0.5346)
